In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import datetime
import calendar
import zipfile

In [2]:
def download_deliverables():
    """
    Downloads the deliverables of the current day.

    deliverables_url = "https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0"

    creates the driver.

    opens the deliverables_url.
    
    downloads the file.

    Methods:
    --------

    create_driver : creates the chrome driver.

    download : downloads the file. invokes extract_save method.

    extract_save : extracts the file and renames it to the specified name.

    """

    deliverables_url = "https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0"

    path = os.path.join(os.getcwd(), os.path.join("Data"))

    if not os.path.exists(path):
        os.makedirs("Data")

    def extract_save(name):
        """
        extracts the file and renames it to the specified name.

        Parameters
        ----------
        name : string
            name of the file i.e, security id

        """

        res = zipfile.ZipFile(os.path.join(path,name))
        res.extractall(path)
        deli = pd.read_csv(os.path.join(path,name),sep="|",converters={'DATE': lambda x: str(x)})
        deli["DATE"] = deli["DATE"].apply(lambda row:datetime.datetime.strptime(str(row),"%d%m%Y").date())
        deli.to_csv(os.path.join(path,"deliverable"+".csv"),index=None)

    def download():
        """
        downloads the file.
        invokes the extract_save method.

        """
        element = driver.find_element_by_xpath('/html/body/form/div[4]/div/div/div[3]/div/div[1]/div[4]/a')
        element.click()
        url = element.get_attribute("href")
        name = url.split("/")[-1]
        time.sleep(3)
        driver.quit()
        extract_save(url.split("/")[-1])
        os.remove(os.path.join(path,name))
        os.remove(os.path.join(path,name.replace(".zip",".txt")))

    def create_driver():
        """
        Creates a Chrome Driver.

        Returns
        --------
        driver : driver
            chrome web driver.
        """
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    driver = create_driver()
    driver.get(deliverables_url)
    download()

In [3]:
def download_bhavcopy():

    """
    Downloads the bhavcopy of the current day.

    bhav_copy_url = "https://www.bseindia.com/markets/MarketInfo/BhavCopy.aspx"

    creates the driver.
    opens the bhav_copy_url.
    sets the current date.
    downloads the file.

    Methods:
    --------
    create_driver : creates the chrome driver.

    set_date : Sets the day, month, year.

    download : downloads the file. invokes extract_save method.

    extract_save : extracts the file and renames it to the specified name.

    """
    path = os.path.join(os.getcwd(), os.path.join("Data"))

    if not os.path.exists(path):
        os.makedirs("Data")

    bhav_copy_url = "https://www.bseindia.com/markets/MarketInfo/BhavCopy.aspx"

    def extract_save(name):
        """
        extracts the file and renames it to the specified name.

        Parameters
        ----------
        name : string
            name of the file i.e, security id

        """
        res = zipfile.ZipFile(os.path.join(path,name))
        res.extractall(path)
        today = datetime.datetime.strptime(str(name[2:-8]),"%d%m%y").date()
        bhav = pd.read_csv(os.path.join(path,name[:8]+".csv"))
        bhav["DATE"] = today
        bhav.to_csv(os.path.join(path,"bhav.csv"),index=None)
        os.remove(os.path.join(path,name[:8]+".csv"))

    def download():
        """
        downloads the file.
        invokes the extract_save method.

        """
        element = driver.find_element_by_xpath('/html/body/form/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/table/tbody/tr/td/table[1]/tbody/tr/td/table/tbody/tr[2]/td[1]/table/tbody/tr/td/ul/li[1]/a')
        element.click()
        time.sleep(3)
        url = element.get_attribute("href")
        driver.quit()
        name = url.split("/")[-1]
        extract_save(name)
        os.remove(os.path.join(path,name))

    def create_driver():
        """
        Creates a Chrome Driver.

        Returns
        --------
        driver : driver
            chrome web driver.
        """
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    driver = create_driver()
    driver.get(bhav_copy_url)
    download()

In [4]:
def convertBhavCopyToStock(bhav,deli):
    """
    creates a new stock data.

    Parameters
    ----------
    bhav : dataframe

    deli : dataframe

    Returns
    -------

    df : dataframe
        stock dataframe

    """
    # bhav = pd.read_csv(os.path.join(BhavCopyPath,name))
    # deli = pd.read_csv(os.path.join(DeliverablesPath,name))
    df = pd.DataFrame()
    bhav["DATE"] = pd.to_datetime(bhav["DATE"])
    df["Date"] = bhav["DATE"].iloc[0]
    df["code"] = bhav["SC_CODE"]
    df["Open Price"] = bhav["OPEN"]
    df["High Price"] = bhav["HIGH"]
    df["Low Price"] = bhav["LOW"]
    df["Close Price"] = bhav["CLOSE"]
    df["WAP"] = bhav["NET_TURNOV"] / bhav["NO_OF_SHRS"]
    df["No.of Shares"] = bhav["NO_OF_SHRS"]
    df["No. of Trades"] = bhav["NO_TRADES"]
    df["Total Turnover (Rs.)"] = bhav["NET_TURNOV"]
    df["Deliverable Quantity"] = deli["DELIVERY QTY"]
    df["% Deli. Qty to Traded Qty"] = deli["DELV. PER."]
    df["Spread High-Low"] = bhav["HIGH"] - bhav["LOW"]
    df["Spread Close-Open"] = bhav["CLOSE"] - bhav["OPEN"]
    df["Unix Date"] = bhav["DATE"].apply(lambda x : time.mktime(x.timetuple()))
    return df

In [ ]:
def update_files():
    path = os.path.join(os.getcwd(), os.path.join("Data"))
    result = pd.read_csv(os.path.join(path, "result.csv"))
    result = result.set_index("Code")
    stockpath = os.path(path, "Stock")
    for index, row in result.iterrows():
        try:
            if os.path.exists(os.path.join(stockpath, str(index)+".csv")):
                stk = pd.read_csv(os.path.join(stockpath, str(index)+".csv"))
                stk.loc[len(stk.index)] = row
                stk["Date"] = pd.to_datetime(stk["Date"])
                stk = stk.drop_duplicates(subset=["Date"], keep="first")
                stk = stk.sort_values(by=["Date"], ascending=[False])
                stk.to_csv(os.path.join(
                    stockpath, str(index)+".csv"), index=None)
        except:
            pass

In [ ]:
download_deliverables()
download_bhavcopy()
path = os.path.join(os.getcwd(), "Data")
bhav = pd.read_csv(os.path.join(path, "bhav.csv"))
deli = pd.read_csv(os.path.join(path, "deliverable.csv"))
result = convertBhavCopyToStock(bhav, deli)
result.to_csv(os.path.join(path, "result.csv"), index=None)
update_files()
